# Project 3: MCMC and Statistical Mechanics

## Write a Python module to run MCMC simulations

Imagine that you are playing a game with your friend in which you roll a 6-sided die.  The way the game works is that every time the die lands on side 1 or side 2, you win one dollar, and every time the die lands on side 3, 4, 5, or 6, your friend wins one dollar.  Because you really want to win the game at all costs, you decide to cheat and use a weighted die.  This means that the side is preferentially weighted so that sides 1 and 2 are each three times more likely to land face up than sides 3, 4, 5, and 6.  In other words, if $p_k$ denotes the probability that side $k$ will land face up, then

\begin{align}
    p_1 = p_2, \qquad p_3 = p_4 = p_5 = p_6 \qquad p_1 = 3 p_3.
\end{align}

Your module should contain a function

    weighted_die(num_steps)

That runs an Markov Chain Monte Carlo (MCMC) simulation to simulate the results of rolling the die above a large number of times.  The argument `num_steps` tells the simulation how many MCMC updates you want to perform.  The simulation should keep track of your winnings (an integer dollar amount) in the game after each update, and it should return the approximate expected value (average) of your earnings in the game.  Note that you should be able to figure out the expected value of your earnings analytically in this case and compare this to what your simulation produces as a sanity check.

Your module should contain a second function

    two-dim-ising(temp, num_steps)
    
that simulates the 2-d ising model on a periodic lattice of a specified size and at a given temperature.  The simulation should return the value of the magnetization M for the given temperature which can then be compared to Onsager's exact result.  The inputs `temp` and `num_steps` specify the desired temperature of the lattice and the number of MCMC updates you wish to perform during the simulation.

## Write a Jupyter notebook that explores and describes the physics of the 2d Ising simulation

Your notebook should apply the function `two-dim-ising` to investigate the physics of the 2d Ising model on a periodic lattice.  In particular, you should:

1. Vertify that for sufficiently large lattices, the simulation produces a magnetization value that agrees with Onsager's theoretical result.
1. Create images of what the lattice looks like at the end of the simulation for various values of the temperature, and comment on in what ways the lattice's qualitative features are different for various temperatures.  Comment, in particular, on how the lattice looks at low temperatures, high temperatures, and temperatures near the critical temperature.